In [1]:
import os
import certifi  # Import the certifi module
os.environ['REQUESTS_CA_BUNDLE'] = certifi.where()
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import requests
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# URL de la página web
url = "https://apps5.mineco.gob.pe/transparencia/mensual/Navegar_4.aspx?_uhc=yes&0=&22=2&8=&y=2010&cpage=1&psize=400"

# Realizar una solicitud GET para obtener el contenido de la página
response = requests.get(url)

In [3]:
df=pd.read_html(url,encoding="utf-8")[8].iloc[:,[1,5]]
df.columns=['Función','2023.1']

In [19]:
import pandas as pd
import requests

# Definir los parámetros de trimestre y año
trimestres = [1, 2, 3, 4]
años = np.arange(2010,2023)

# Inicializar una lista para almacenar los DataFrames
dfs = []

# Iterar a través de los trimestres y años
for trimestre in trimestres:
    for a in años:
        # Construir la URL con los parámetros de trimestre y año
        url = f"https://apps5.mineco.gob.pe/transparencia/mensual/Navegar_4.aspx?_uhc=yes&0=&22={trimestre}&8=&y={a}&cpage=1&psize=400"

        # Realizar una solicitud GET para obtener el contenido de la página
        response = requests.get(url)

        # Leer la tabla relevante del HTML
        df = pd.read_html(response.text, encoding="utf-8")[8]

        # Seleccionar las columnas 'Función' y '2023.1'
        df = df.iloc[:, [1, 5]]
        df.columns = ['Función', f'{a}.{trimestre}']

        # Agregar el DataFrame a la lista
        dfs.append(df)

# Concatenar los DataFrames en uno solo
df_final = pd.concat(dfs, axis=1)

In [33]:
# Inicializa el DataFrame final con el primer DataFrame de la lista
df_final = dfs[0]

# Realiza un bucle para unir horizontalmente los DataFrames restantes
for i in range(1, len(dfs)):
    df_final = pd.merge(df_final, dfs[i], on="Función", how="inner")

In [34]:
df_raw=df_final.T.reset_index()

In [35]:
df_raw.columns = df_raw.iloc[0,:]

In [36]:
df_raw.columns

Index(['Función', '01: LEGISLATIVA', '02: RELACIONES EXTERIORES',
       '03: PLANEAMIENTO, GESTION Y RESERVA DE CONTINGENCIA',
       '04: DEFENSA Y SEGURIDAD NACIONAL', '05: ORDEN PUBLICO Y SEGURIDAD',
       '06: JUSTICIA', '07: TRABAJO', '08: COMERCIO', '09: TURISMO',
       '10: AGROPECUARIA', '11: PESCA', '12: ENERGIA', '13: MINERIA',
       '14: INDUSTRIA', '15: TRANSPORTE', '16: COMUNICACIONES',
       '18: SANEAMIENTO', '19: VIVIENDA Y DESARROLLO URBANO', '20: SALUD',
       '21: CULTURA Y DEPORTE', '22: EDUCACION', '23: PROTECCION SOCIAL',
       '24: PREVISION SOCIAL', '25: DEUDA PUBLICA'],
      dtype='object', name=0)

In [37]:
abreviaciones = {
    'Función': 'funci',
    '01: LEGISLATIVA': 'legis',
    '02: RELACIONES EXTERIORES': 'relex',
    '03: PLANEAMIENTO, GESTION Y RESERVA DE CONTINGENCIA': 'plage',
    '04: DEFENSA Y SEGURIDAD NACIONAL': 'defse',
    '05: ORDEN PUBLICO Y SEGURIDAD': 'ordpu',
    '06: JUSTICIA': 'justi',
    '07: TRABAJO': 'traba',
    '08: COMERCIO': 'comer',
    '09: TURISMO': 'turis',
    '10: AGROPECUARIA': 'agrop',
    '11: PESCA': 'pesca',
    '12: ENERGIA': 'energ',
    '13: MINERIA': 'minee',
    '14: INDUSTRIA': 'indus',
    '15: TRANSPORTE': 'trans',
    '16: COMUNICACIONES': 'comun',
    '18: SANEAMIENTO': 'sanea',
    '19: VIVIENDA Y DESARROLLO URBANO': 'vived',
    '20: SALUD': 'salud',
    '21: CULTURA Y DEPORTE': 'cultu',
    '22: EDUCACION': 'educa',
    '23: PROTECCION SOCIAL': 'prots',
    '24: PREVISION SOCIAL': 'previ',
    '25: DEUDA PUBLICA': 'deudp'
}

In [38]:
df_raw=df_raw.drop(0).sort_values(by="Función", ascending=True).reset_index(drop=True)
df_raw.columns=abreviaciones.values()
df_raw

,funci,legis,relex,plage,defse,ordpu,justi,traba,comer,turis,...,trans,comun,sanea,vived,salud,cultu,educa,prots,previ,deudp
0,2010.1,57216643,130985415,1994493386,745186012,1095025119,452984801,46086863,85137986,43365868,...,1873536392,38897189,258146200,227974867,1398058676,185347093,2736374731,509490629,2310397655,2127955783
1,2010.2,61576159,201868549,2025439617,857799073,1030436038,536025494,61329243,97012581,55950700,...,3256506236,53714016,502870817,244203994,1688217477,256866743,2913067872,638903898,2427112586,1999518534
2,2010.3,71322736,125343393,2289494099,820532532,1221560794,693738338,76927005,112082734,60800959,...,2958929108,52977025,882134287,214792263,1556739944,313719951,3198376369,908494405,2933972233,2170570287
3,2010.4,84713492,139003240,2936503412,1207389117,1329516016,625786269,125162622,112588615,101009064,...,4470509794,126830584,1184170794,337849526,2108772308,600976326,3884784858,1141929141,2936285355,1916737610
4,2011.1,67681134,126393584,1754178927,806262114,1052289761,488233217,42342998,50858413,33938112,...,1651959831,112203040,310050791,140183191,1435729564,146869985,2791086141,542929976,2932317745,2071788028
5,2011.2,64359370,129470939,2294092034,736853670,1077610756,571633328,32320881,68647708,50603564,...,3074863399,46812049,486072896,182312015,1609071616,258426676,3025838187,692553337,2603228571,1560151479
6,2011.3,89154872,138500460,2637550503,886266633,1245744844,623942257,28641158,76238079,60297355,...,2867503720,56709854,702115490,216856279,1742171900,313042638,3434585292,787359877,2880908183,2596709734
7,2011.4,90499453,156128684,4215012109,1035952936,1516036054,833581565,103350827,124645322,142224504,...,4788955185,88464960,1785476737,389913545,2993810569,448112531,4497620883,992941595,3222153897,1787001877
8,2012.1,80758422,132595811,2430637018,760869269,1195703332,660700738,57353995,99318590,78117701,...,1748035336,43871292,587135885,140690506,1742864536,216041754,3218288137,856735362,2809714796,3015697346
9,2012.2,68322407,120558043,2765081637,1109417338,1398714055,597208668,77639070,95992580,85270708,...,3013232761,48584817,842440560,304780928,1938949465,284181133,3331605062,868976535,2529216643,1585889263


In [39]:
df_raw.to_excel('archivo_excel.xlsx', index=False)